# » `Dependencias`:

In [ ]:
# !pip install plotly
# !pip install sqlalchemy
# !pip install numpy
# !pip install matplotlib

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import re
# import cufflinks as cf
# from IPython.display import display, HTML

# cf.set_config_file

## 1. Conección → SIM(172.27.0.124)

In [3]:
# spring.datasource.url = jdbc:sqlserver://172.27.250.27;databaseName=SIRIM
SERVER = '172.27.0.242' #'172.27.0.124'
#DRIVER = 'SQL Server Native Client 11.0'
DRIVER = 'ODBC Driver 17 for SQL Server'
DATABASE = 'SIM'
USERNAME = 'udesa' # userestadistica
PASSWORD = 'DESARROLLO2006' # $Us3R_3sT4d1sTic4$
DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

## 2. Métodos genéricos:

In [4]:
def get_query_sql(query):
  try:
    df = pd.read_sql(query, connection)
    return df
  except:
    print('¡Ocurrió un error!')

## 3. EXTRACCIÓN Y EXPLORACIÓN DE DATOS:

### 3.1 Aerolineas `Google, SIM y STSG` ...

In [5]:
df_aerolineas_g = pd.read_excel('./data/aerolineas.xlsx')
df_et_atsg = pd.read_excel('./data/aerolineas_atsg.xlsx')

In [8]:
SQL_QUERY_2 = ''' 
                  SELECT 
                     t.nIdTransportista, t.sNombreRazon
                  FROM (

                     SELECT 
                        DISTINCT mm.nIdTransportista
                     FROM SimMovMigra mm
                     WHERE 
                        mm.bAnulado = 0 AND mm.bTemporal = 0
                        AND mm.dFechaControl BETWEEN '2023-01-01 00:00:00.000' AND '2023-12-31 23:59:59.999'

                  ) mm2 JOIN SimEmpTransporte t ON mm2.nIdTransportista = t.nIdTransportista
                  WHERE
                     t.sIdViaTransporte = 'A'  -- A | AEREO

'''

df_et_sim = get_query_sql(SQL_QUERY_2)

In [10]:
# len(df_et_atsg)
# renombra columnas para join
df_aerolineas_g.rename({ 'Código ICAO': 'ICAO', 'Código IATA': 'IATA' }, axis=1, inplace=True)

# join
df_aereol_g_join_et_atsg = df_aerolineas_g.merge(df_et_atsg, how='right', on='ICAO').dropna(axis=0, how='any')
# df_aereol_g_join_et_atsg

In [16]:
# sNombreRazon: Aerolínea
df_et_sim.rename({ 'sNombreRazon': 'Aerolínea' }, axis=1, inplace=True)

# 41 aerolineas Registradas en SIM
df_join_final = df_et_sim.merge(df_aereol_g_join_et_atsg, how='left', on='Aerolínea')
df_join_final


,nIdTransportista,Aerolínea,IATA_x,ICAO,Código 3 Dígitos,País,Code,Name,Active,IATA_y
0,4,AIR FRANCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,AER. ARGENTINAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34,LAN PERU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,LACSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,53,UNITED AIRLINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,59,SPIRIT AIRLINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,62,BOLIVIANA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,76,LAW - LATIN AMERICAN WINGS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,A.AIRLINES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,27,ECUATORIANA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
